In [2]:
import sys

import pandas as pd

sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
from sem_covid import config
import json
import numpy as np
import re
from langdetect import detect, detect_langs, DetectorFactory

DetectorFactory.seed = 0

from sem_covid.services.store_registry import StoreRegistry, store_registry


In [41]:
print(2)

2


In [3]:
es_store = store_registry.es_index_store()



In [4]:
# create model for document embeddings
from sem_covid.services.model_registry import embedding_registry

emb_model = embedding_registry.sent2vec_universal_sent_encoding()


INFO:absl:Using /tmp/tfhub_modules to cache modules.


In [ ]:
# create model for topic embeddings
import mlflow

EXPERIMENT_ID = '101'
BUCKET_NAME = 'mlflow'

best_run = mlflow.search_runs(
    experiment_ids=EXPERIMENT_ID,
    order_by=['parameters.freq_topic_minus_1 ASC']
).iloc[0]



bert_model = store_registry.minio_feature_store(BUCKET_NAME).get_features(
    features_name=f'{EXPERIMENT_ID}/{best_run.run_id}/artifacts/model/model.pkl')



In [5]:
def make_target_group_l1_column(dataset: pd.DataFrame):
    for col in TARGET_GROUPS_L1:
        dataset[col]=dataset[col].apply(lambda x: col if x == 1 else "" )

    dataset["pwdb_target_group_l1"] = dataset[TARGET_GROUPS_L1].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)\
        .apply(lambda x: x.split())


In [6]:
def make_document_embeddings_column(dataset: pd.DataFrame):
    dataset["document_embeddings"] = emb_model.encode(dataset.content.values)


In [ ]:
def make_topic_embeddings_column(dataset: pd.DataFrame):
    bert_probabilities = bert_model.transform(documents=dataset['content'],
                                              embeddings=np.array(dataset["document_embeddings"].values.tolist()).astype("float32"))[1]
    dataset['topic_based_embedding'] = bert_probabilities.tolist()

In [7]:
def create_new_column_with_defined_value(dataset: pd.DataFrame, column_name: str, value=None, empty_array=False):
    if empty_array:
       dataset[column_name] = [np.empty(0,dtype=float)]*len(dataset)
    else:
        dataset[column_name] = value


In [8]:
TARGET_GROUPS_L1 = ["businesses", "workers", "citizens"]
COMMON_DATASET_COLUMNS = ["title","content","date","doc_source","country","pwdb_category",
                          "pwdb_target_group_l1","pwdb_funding", "pwdb_type_of_measure",
                          "pwdb_actors", "document_embeddings"]
# to add topic_embeddings column
SPECIFIC_DATASET_COLUMNS = ["eu_cellar_subject_matter_labels", "eu_cellar_resource_type_labels","eu_cellar_directory_code_labels",
                            "eu_cellar_author_labels", "pwdb_target_group_l2", "ireland_keyword", "ireland_department_data",
                            "ireland_campaign", "ireland_page_type", "eu_timeline_topic"]

In [45]:
#Load dataframes from elastic
pwdb_df = es_store.get_dataframe(index_name=config.PWDB_ELASTIC_SEARCH_INDEX_NAME)
eu_cellar_df = es_store.get_dataframe(index_name=config.EU_CELLAR_ELASTIC_SEARCH_INDEX_NAME + "_enriched")
eu_timeline_df = es_store.get_dataframe(index_name=config.EU_TIMELINE_ELASTIC_SEARCH_INDEX_NAME + "_enriched")
ir_timeline_df = es_store.get_dataframe(index_name=config.IRELAND_TIMELINE_ELASTIC_SEARCH_INDEX_NAME + "_enriched")



100% (1381 of 1381) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (3175 of 3175) |####################| Elapsed Time: 0:00:01 Time:  0:00:01
100% (225 of 225) |######################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1921 of 1921) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [10]:
#textual columns in PWDB dataset into one content column
textual_columns = [col for col in pwdb_df.columns if "description" in col]
textual_columns.insert(0,"title")
pwdb_df["content"] = pwdb_df[textual_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
pwdb_df["content"]

_id
tika/2ec585e04df9d361a951e7d26ce5d3ab06e4d17612e36ee84696c1451c972c4b    Temporary layoff scheme and reimbursement of r...
tika/497730ad75916dd4a3174469d0946ccf6218adfd9513cfffb52038a3dce3fa8d    Protocol for the containment of the COVID-19 s...
tika/5f74df01de688c3083e10eb50ebd865b081ee3595a662118c3d1d4ca1ad09bc2    ELY centres' business development aid to micro...
tika/ae1904ac24001af20cfaf9ce898605e95122371e26ae96b7337d114098819c33    Regularisation of immigrants' presence on nati...
tika/3bafed913ae25dd3152fec477a007abfa079507146ee1b7f8b0a9abad10dc2aa    Economic reactivation measures to face the imp...
                                                                                               ...                        
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f    Government support to assist the sports sector...
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6    National public sector collective agreement 20...
tika/3df93a8

In [11]:
make_target_group_l1_column(pwdb_df)
pwdb_df["pwdb_target_group_l1"]

_id
tika/2ec585e04df9d361a951e7d26ce5d3ab06e4d17612e36ee84696c1451c972c4b              []
tika/497730ad75916dd4a3174469d0946ccf6218adfd9513cfffb52038a3dce3fa8d    [businesses]
tika/5f74df01de688c3083e10eb50ebd865b081ee3595a662118c3d1d4ca1ad09bc2       [workers]
tika/ae1904ac24001af20cfaf9ce898605e95122371e26ae96b7337d114098819c33      [citizens]
tika/3bafed913ae25dd3152fec477a007abfa079507146ee1b7f8b0a9abad10dc2aa    [businesses]
                                                                             ...     
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f    [businesses]
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6       [workers]
tika/3df93a8fb254187e8783d19760b465a98d88863e59a513b35a4d8062248e4190       [workers]
tika/71fc1cad3a3a8b9de4da7080acc3a8251fa8ff8c5b7514f082f18c1cf69ab140       [workers]
tika/417384d22c3c6f1768aed95a928111a4652fd832c8d5ee38594ca661e53488ca              []
Name: pwdb_target_group_l1, Length: 1381, dtype: o

In [12]:
make_document_embeddings_column(pwdb_df)
pwdb_df["document_embeddings"]

_id
tika/2ec585e04df9d361a951e7d26ce5d3ab06e4d17612e36ee84696c1451c972c4b    [-0.05382201448082924, -0.054840292781591415, ...
tika/497730ad75916dd4a3174469d0946ccf6218adfd9513cfffb52038a3dce3fa8d    [0.043122436851263046, -0.008066767826676369, ...
tika/5f74df01de688c3083e10eb50ebd865b081ee3595a662118c3d1d4ca1ad09bc2    [-0.011736772023141384, 0.05330539867281914, 0...
tika/ae1904ac24001af20cfaf9ce898605e95122371e26ae96b7337d114098819c33    [-0.020241111516952515, -0.05406587943434715, ...
tika/3bafed913ae25dd3152fec477a007abfa079507146ee1b7f8b0a9abad10dc2aa    [0.020759258419275284, 0.01490385364741087, 0....
                                                                                               ...                        
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f    [-0.0541599802672863, -0.05460011959075928, -0...
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6    [0.014643954113125801, -0.05534055083990097, 0...
tika/3df93a8

In [13]:
#pwdb_df["doc_source"] = "ds_pwdb"
create_new_column_with_defined_value(pwdb_df, "doc_source", "ds_pwdb")


In [14]:
# create specific dataset columns

for column in SPECIFIC_DATASET_COLUMNS:
    create_new_column_with_defined_value(pwdb_df,column_name=column,empty_array=True)

pwdb_df[SPECIFIC_DATASET_COLUMNS]



,eu_cellar_subject_matter_labels,eu_cellar_resource_type_labels,eu_cellar_directory_code_labels,eu_cellar_author_labels,pwdb_target_group_l2,ireland_keyword,ireland_department_data,ireland_campaign,ireland_page_type,eu_timeline_topic
_id,,,,,,,,,,
tika/2ec585e04df9d361a951e7d26ce5d3ab06e4d17612e36ee84696c1451c972c4b,[],[],[],[],[],[],[],[],[],[]
tika/497730ad75916dd4a3174469d0946ccf6218adfd9513cfffb52038a3dce3fa8d,[],[],[],[],[],[],[],[],[],[]
tika/5f74df01de688c3083e10eb50ebd865b081ee3595a662118c3d1d4ca1ad09bc2,[],[],[],[],[],[],[],[],[],[]
tika/ae1904ac24001af20cfaf9ce898605e95122371e26ae96b7337d114098819c33,[],[],[],[],[],[],[],[],[],[]
tika/3bafed913ae25dd3152fec477a007abfa079507146ee1b7f8b0a9abad10dc2aa,[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f,[],[],[],[],[],[],[],[],[],[]
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6,[],[],[],[],[],[],[],[],[],[]
tika/3df93a8fb254187e8783d19760b465a98d88863e59a513b35a4d8062248e4190,[],[],[],[],[],[],[],[],[],[]


In [15]:
#add values if the specific column exists in the current dataset
pwdb_df["pwdb_target_group_l2"] = pwdb_df["target_groups"]
pwdb_df["pwdb_target_group_l2"]


_id
tika/2ec585e04df9d361a951e7d26ce5d3ab06e4d17612e36ee84696c1451c972c4b                                                   []
tika/497730ad75916dd4a3174469d0946ccf6218adfd9513cfffb52038a3dce3fa8d                   [Sector specific set of companies]
tika/5f74df01de688c3083e10eb50ebd865b081ee3595a662118c3d1d4ca1ad09bc2                                      [Self-employed]
tika/ae1904ac24001af20cfaf9ce898605e95122371e26ae96b7337d114098819c33                                 [Migrants, Refugees]
tika/3bafed913ae25dd3152fec477a007abfa079507146ee1b7f8b0a9abad10dc2aa                   [Sector specific set of companies]
                                                                                               ...                        
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f                   [Sector specific set of companies]
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6                            [Other groups of workers]
tika/3df93a8

In [16]:
# rename columns in the current dataset and prepare new version for merging
pwdb_df.rename(columns={"category": "pwdb_category", "funding": "pwdb_funding", "type_of_measure": "pwdb_type_of_measure"
    , "actors": "pwdb_actors", "start_date": "date"}, inplace=True)

tmp_pwdb_df = pd.DataFrame(pwdb_df[COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS])

In [17]:
# Verify if the new version has all the necessary columns
if tmp_pwdb_df.columns.tolist() == COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS:
    print("ready for merge")
else:
    raise ValueError("Some columns are missing!! Not ready for merge")

ready for merge


In [323]:
tmp_pwdb_df["document_embeddings"]


_id
tika/2ec585e04df9d361a951e7d26ce5d3ab06e4d17612e36ee84696c1451c972c4b    [-0.05382201448082924, -0.054840292781591415, ...
tika/497730ad75916dd4a3174469d0946ccf6218adfd9513cfffb52038a3dce3fa8d    [0.043122436851263046, -0.008066767826676369, ...
tika/5f74df01de688c3083e10eb50ebd865b081ee3595a662118c3d1d4ca1ad09bc2    [-0.011736772023141384, 0.05330539867281914, 0...
tika/ae1904ac24001af20cfaf9ce898605e95122371e26ae96b7337d114098819c33    [-0.020241111516952515, -0.05406587943434715, ...
tika/3bafed913ae25dd3152fec477a007abfa079507146ee1b7f8b0a9abad10dc2aa    [0.020759258419275284, 0.01490385364741087, 0....
                                                                                               ...                        
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f    [-0.0541599802672863, -0.05460011959075928, -0...
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6    [0.014643954113125801, -0.05534055083990097, 0...
tika/3df93a8

In [18]:
import mlflow

EXPERIMENT_ID = '101'
BUCKET_NAME = 'mlflow'

best_run = mlflow.search_runs(
    experiment_ids=EXPERIMENT_ID,
    order_by=['parameters.freq_topic_minus_1 ASC']
).iloc[0]



bert_model = store_registry.minio_feature_store(BUCKET_NAME).get_features(
    features_name=f'{EXPERIMENT_ID}/{best_run.run_id}/artifacts/model/model.pkl')


In [19]:
tmp_pwdb_df["document_embeddings"]

# type(tmp_pwdb_df["document_embeddings"][0])

_id
tika/2ec585e04df9d361a951e7d26ce5d3ab06e4d17612e36ee84696c1451c972c4b    [-0.05382201448082924, -0.054840292781591415, ...
tika/497730ad75916dd4a3174469d0946ccf6218adfd9513cfffb52038a3dce3fa8d    [0.043122436851263046, -0.008066767826676369, ...
tika/5f74df01de688c3083e10eb50ebd865b081ee3595a662118c3d1d4ca1ad09bc2    [-0.011736772023141384, 0.05330539867281914, 0...
tika/ae1904ac24001af20cfaf9ce898605e95122371e26ae96b7337d114098819c33    [-0.020241111516952515, -0.05406587943434715, ...
tika/3bafed913ae25dd3152fec477a007abfa079507146ee1b7f8b0a9abad10dc2aa    [0.020759258419275284, 0.01490385364741087, 0....
                                                                                               ...                        
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f    [-0.0541599802672863, -0.05460011959075928, -0...
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6    [0.014643954113125801, -0.05534055083990097, 0...
tika/3df93a8

In [39]:
bert_probabilities = bert_model.transform(documents=tmp_pwdb_df['content'], embeddings=np.array(tmp_pwdb_df["document_embeddings"].values.tolist()).astype("float32"))[1]
tmp_pwdb_df['topic_based_embedding'] = bert_probabilities.tolist()


In [40]:
tmp_pwdb_df['topic_based_embedding']

_id
tika/2ec585e04df9d361a951e7d26ce5d3ab06e4d17612e36ee84696c1451c972c4b    [0.001350240214914433, 0.00021572137051582428,...
tika/497730ad75916dd4a3174469d0946ccf6218adfd9513cfffb52038a3dce3fa8d    [2.5057711177400634e-08, 2.790581625758431e-09...
tika/5f74df01de688c3083e10eb50ebd865b081ee3595a662118c3d1d4ca1ad09bc2    [4.069156225100791e-12, 1.0782121587005009e-12...
tika/ae1904ac24001af20cfaf9ce898605e95122371e26ae96b7337d114098819c33    [0.0002691530960288354, 3.1798562077396803e-06...
tika/3bafed913ae25dd3152fec477a007abfa079507146ee1b7f8b0a9abad10dc2aa    [0.8514289137347775, 9.008173699930881e-80, 1....
                                                                                               ...                        
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f    [1.6434957524531868e-16, 1.0385811220730888e-1...
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6    [0.026078051020881, 0.024558885443965266, 0.01...
tika/3df93a8

In [29]:
np.array(tmp_pwdb_df["document_embeddings"].values.tolist())[0]


array([-0.05382201, -0.05484029, -0.00939105,  0.03606156, -0.02999368,
        0.00056277,  0.01166629, -0.04972555,  0.04544508, -0.05484084,
       -0.05408668,  0.02478002,  0.04701047, -0.0093147 , -0.00444791,
       -0.05482116, -0.05476168,  0.01637614, -0.05382964,  0.05461138,
       -0.05434485,  0.02192224,  0.05483181, -0.05231631, -0.00100611,
        0.05478474, -0.05483991,  0.02949943,  0.03400396,  0.03548795,
        0.05410358,  0.05062882,  0.05480984,  0.05316053, -0.0544694 ,
       -0.03905037, -0.04640208, -0.05075841,  0.01201403, -0.03234813,
       -0.00696255,  0.00849448, -0.01948846, -0.04986804, -0.04172122,
       -0.0527373 ,  0.02757298,  0.05288697, -0.01026675, -0.05481278,
       -0.05477301,  0.05395877, -0.05449239,  0.01846773,  0.0527554 ,
       -0.04101624, -0.0543223 ,  0.04985623,  0.05415764, -0.05463103,
        0.05148248,  0.03353117, -0.04902823, -0.05197769, -0.05073191,
       -0.05343049,  0.05280374,  0.04891632, -0.00189262,  0.04

In [35]:
set(map(len,tmp_pwdb_df['document_embeddings'].values.tolist()))

{2051,
 2052,
 2054,
 2055,
 4103,
 2057,
 2056,
 2058,
 2060,
 2061,
 2063,
 2065,
 2066,
 2068,
 2069,
 2071,
 2072,
 2073,
 2074,
 2078,
 2081,
 2083,
 2084,
 2088,
 2094,
 2096,
 2097,
 2100,
 4149,
 2102,
 2103,
 2105,
 2106,
 2107,
 2111,
 2115,
 2116,
 4164,
 2120,
 2123,
 4173,
 2126,
 2128,
 4178,
 2131,
 2132,
 2133,
 4182,
 4184,
 2137,
 2139,
 4188,
 2141,
 2142,
 2143,
 4192,
 2144,
 2146,
 2147,
 2148,
 2149,
 2151,
 2153,
 2154,
 2155,
 2157,
 2158,
 2159,
 2161,
 2162,
 2163,
 4210,
 2164,
 2171,
 2173,
 2175,
 2177,
 2178,
 4227,
 4226,
 2181,
 2182,
 2183,
 2184,
 2186,
 2187,
 2188,
 4234,
 2190,
 2191,
 2194,
 2195,
 2196,
 2199,
 2202,
 2203,
 2204,
 2206,
 2207,
 2208,
 2209,
 2210,
 2213,
 2214,
 2217,
 2219,
 4269,
 6318,
 2223,
 2224,
 2225,
 2222,
 2227,
 2230,
 2231,
 2232,
 2234,
 2235,
 2236,
 2240,
 4289,
 2241,
 2246,
 2250,
 2251,
 2252,
 2254,
 2256,
 2257,
 2258,
 2260,
 2261,
 2262,
 2263,
 2265,
 2266,
 2267,
 4314,
 2269,
 2272,
 2273,
 2276,
 2279,

In [37]:
set(tmp_pwdb_df["content"].apply(lambda x: len(x) < 10))


{False}

In [ ]:
# tmp_pwdb_df["document_embeddings"] = emb_model.encode(tmp_pwdb_df.content.values)
# tmp_pwdb_df["document_embeddings"]
#


In [ ]:
# # Create new dataframe with the wanted content and columns from PWDB dataset
# tmp_pwdb_df = pd.DataFrame(pwdb_df[["creation_date", "title", "merged_content"]])
# tmp_pwdb_df["doc_source"] = "pwdb"


# # Print only textual columns
# pwdb_df[textual_columns]
#
# #Merge content of the textual columns into one new column
# pwdb_df["merged_content"] = pwdb_df[textual_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
# pwdb_df["merged_content"]
#
# #What are the columns that shows dates
# date_columns = [col for col in pwdb_df.columns if "date" in col]
# pwdb_df[date_columns]
#
# # Create new dataframe with the wanted content and columns from PWDB dataset
# tmp_pwdb_df = pd.DataFrame(pwdb_df[["creation_date", "title", "merged_content"]])
# tmp_pwdb_df["doc_source"] = "pwdb"
#
# # See the newly created PWDB dataframe
# tmp_pwdb_df

In [234]:
# Print dataframe columns
eu_cellar_df.columns



Index(['work', 'title', 'cdm_types', 'cdm_type_labels', 'resource_types',
       'resource_type_labels', 'eurovoc_concepts', 'eurovoc_concept_labels',
       'subject_matters', 'subject_matter_labels', 'directory_codes',
       'directory_codes_labels', 'celex_numbers', 'legal_elis', 'id_documents',
       'same_as_uris', 'authors', 'author_labels', 'full_ojs', 'oj_sectors',
       'internal_comments', 'is_in_force', 'dates_document', 'dates_created',
       'legal_dates_entry_into_force', 'legal_dates_signature', 'manifs_pdf',
       'manifs_html', 'pdfs_to_download', 'htmls_to_download', 'dossiers',
       'related_works', 'work_sequences', 'eu_cellar_core',
       'eu_cellar_extended', 'metadata', 'content_path', 'content', 'language',
       'businesses', 'citizens', 'workers', 'category', 'subcategory',
       'type_of_measure', 'funding'],
      dtype='object')

In [315]:
# create columns
create_new_column_with_defined_value(eu_cellar_df,"doc_source", "ds_eu_cellar")
create_new_column_with_defined_value(eu_cellar_df,"country", "European Union")
create_new_column_with_defined_value(eu_cellar_df,"pwdb_actors", "EU (Council, EC, EP)")
make_target_group_l1_column(eu_cellar_df)
eu_cellar_df["date"] = eu_cellar_df["dates_document"]
eu_cellar_df["content"] = eu_cellar_df["title"].replace({None: " "}) + eu_cellar_df["content"].replace({None: " "})
# eu_cellar_df["content"].replace({np.nan: " "})
make_document_embeddings_column(eu_cellar_df)

# create specific dataset columns

for column in SPECIFIC_DATASET_COLUMNS:
    create_new_column_with_defined_value(eu_cellar_df,column_name=column,empty_array=True)

eu_cellar_df[SPECIFIC_DATASET_COLUMNS]


KeyboardInterrupt: 

In [311]:
eu_cellar_specific_columns = ["subject_matter_labels",
                              "resource_type_labels",
                              "directory_codes_labels",
                              "author_labels"]
for column_name in eu_cellar_specific_columns:
    eu_cellar_df["eu_cellar_" + column_name] = eu_cellar_df[column_name]
    eu_cellar_df["eu_cellar_" + column_name] = eu_cellar_df["eu_cellar_" + column_name].apply(lambda x: x if x else [])


,eu_cellar_subject_matter_labels,eu_cellar_resource_type_labels,eu_cellar_directory_code_labels,eu_cellar_author_labels,pwdb_target_group_l2,ireland_keyword,ireland_department_data,ireland_campaign,ireland_page_type,eu_timeline_topic
_id,,,,,,,,,,
0,"[Competition, Concentrations between undertaki...",[Announcements],[],"[Directorate-General for Competition, European...",[],[],[],[],[],[]
1,"[Commercial policy, Dumping]",[Staff working document],[],"[Directorate-General for Trade, European Commi...",[],[],[],[],[],[]
2,"[Freedom of establishment, Internal market - P...",[Legislative resolution],[],"[Committee on Economic and Monetary Affairs, E...",[],[],[],[],[],[]
3,[Economic policy],[Opinion],[],[European Court of Auditors],[],[],[],[],[],[]
4,"[Area of freedom, security and justice, Border...",[Recommendation],[],[Council of the European Union],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...
3170,[],[Judicial information],[],[Court of Justice],[],[],[],[],[],[]
3171,[],[Announcements],[],[Directorate-General for Research and Innovati...,[],[],[],[],[],[]
3172,[],"[Corrigendum, Regulation]",[],"[Directorate-General for Translation, European...",[],[],[],[],[],[]


In [316]:
tmp_eu_cellar_df = pd.DataFrame(eu_cellar_df[COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS])

KeyError: "['pwdb_category', 'pwdb_funding', 'pwdb_type_of_measure'] not in index"

_id
0       [Competition, Concentrations between undertaki...
1                            [Commercial policy, Dumping]
2       [Freedom of establishment, Internal market - P...
3                                       [Economic policy]
4       [Area of freedom, security and justice, Border...
                              ...                        
3170                                                   []
3171                                                   []
3172                                                   []
3173    [Public health, Research and technological dev...
3174                                                   []
Name: eu_cellar_subject_matter_labels, Length: 3175, dtype: object

In [ ]:
# Verify if the new version has all the necessary columns
if tmp_eu_cellar_df.columns.tolist() == COMMON_DATASET_COLUMNS + SPECIFIC_DATASET_COLUMNS:
    print("ready for merge")
else:
    raise ValueError("Some columns are missing!! Not ready for merge")


In [242]:
#What are the columns that shows dates
date_colums = [col for col in eu_cellar_df.columns if "date" in col]
eu_cellar_df[date_colums]


,dates_document,dates_created,legal_dates_entry_into_force,legal_dates_signature
_id,,,,
0,2020-01-30,None,None,None
1,2021-08-30,None,None,None
2,2020-06-18,None,None,None
3,2020-09-07,None,None,None
4,2021-05-20,None,None,None
...,...,...,...,...
3170,2020-12-10,None,None,None
3171,2021-02-24,None,None,None
3172,2020-12-29,None,None,None


In [13]:
# Create new dataframe with the wanted content and columns from Cellar dataset
tmp_cellar_df = pd.DataFrame(eu_cellar_df[["dates_document", "title", "content"]])
tmp_cellar_df["doc_source"] = "eu_cellar"

In [14]:
# See the newly created Cellar dataframe
tmp_cellar_df


,dates_document,title,content,doc_source
_id,,,,
0e3d35dcad11d1d80f8fb1c35599be27fb3e864874ab1ac679fcd004723bc2e3,2020-01-30,Prior notification of a concentration (Case M....,30.1.2020 EN Official Journal of the European ...,eu_cellar
39dac43e4fffa3fc60906f3563e4d1547aea4a0f94e17cd28ec144b117e25df6,2021-08-30,COMMISSION STAFF WORKING DOCUMENT […] Accompan...,"EUROPEAN COMMISSION Brussels, 30.8.2021 SWD(20...",eu_cellar
00f65d205b8df943ae8acbd34b729c3da11da1646e7e324a6c517f95f950a5d9,2020-06-18,P9_TA(2020)0157 Amending Regulations (EU) No 5...,8.9.2021 EN Official Journal of the European U...,eu_cellar
002086bd15c9aba8b8b3cdf88498e25735cf66043ddab8877aca317256615aff,2020-09-07,Opinion No 6/2020 (pursuant to Article 287(4) ...,20.10.2020 EN Official Journal of the European...,eu_cellar
014589ceb95203c13ee4bd097ddf2f164657cb6aeb1650db280b2d8c7a57977d,2021-05-20,Council Recommendation (EU) 2021/816 20 May 20...,21.5.2021 EN Official Journal of the European ...,eu_cellar
...,...,...,...,...
5d1566cde8e0ff00d5e907e551d9202c714e320039c29609324e0eb579395aa1,2020-12-10,Case C-774/19: Judgment of the Court (Sixth Ch...,15.2.2021 EN Official Journal of the European ...,eu_cellar
f63ec4df58c38a804c983d295869f22e89515ef32c686841d43b7a9618e106d9,2021-02-24,Calls for proposals and related activities und...,24.2.2021 EN Official Journal of the European ...,eu_cellar
5c798a5a6aeb16a370e018faa10f1f79ce040d1adc8e0a2c4d06470a3d20f52b,2020-12-29,None,,eu_cellar


In [15]:
# Print dataframe columns
eu_timeline_df.columns

Index(['month_name', 'date', 'title', 'abstract', 'presscorner_links',
       'all_links', 'detail_link', 'detail_type', 'detail_date',
       'detail_location', 'detail_content', 'detail_title', 'detail_pdf_link',
       'press_contacts', 'topics', 'for_more_information_links'],
      dtype='object')

In [16]:
#What are the columns that shows dates
date_columns = [col for col in eu_timeline_df.columns if "date" in col]
eu_timeline_df[date_columns]


,date,detail_date
_id,,
01dc34ca5f6bfa5316012d6f89a4256b8d2ce6e3a12210b0406903dd15bf36ac,2021-01-28,NaN
04d776c061243576ffc1e1c5a724d41ecca2ed12339b2ff473c5aa2ff1a071c7,2021-06-22,2021-06-22
04eb55005a803c7353e5d5ed339d7f3a72cc8da6e49f2b8b152bf8fb40a1398f,2020-06-16,2020-06-17
073e22ec2b44a5f2f6289f09dc954ecd6ca6cbb4a4b33ffd88c3c150572569c0,2020-04-15,2020-04-15
0a3650bef3282d09d971dd92dc01570b7ecbdb67a4bdd0ff1c8cf8443a06b937,2020-12-18,NaN
...,...,...
fa793c764aaf89c283f747036c9d73fd66f0a7e3692be56b8ac985610bef2417,2021-06-29,2021-06-29
fb9c4d1634da94c73fd529b44ec40057c28a5c925a8b8e71d73c49e5a815784d,2020-03-13,2020-03-13
fbad1d421cf80d3e0e4c6b6d3d60a2a6ec8bed4ee19e07cd15d963fa74352420,2021-01-28,2020-03-19


In [44]:
eu_timeline_df["detail_content"]

_id
01dc34ca5f6bfa5316012d6f89a4256b8d2ce6e3a12210b0406903dd15bf36ac                                                  NaN
04d776c061243576ffc1e1c5a724d41ecca2ed12339b2ff473c5aa2ff1a071c7    The European Commission has today adopted a po...
04eb55005a803c7353e5d5ed339d7f3a72cc8da6e49f2b8b152bf8fb40a1398f    Today, to help protect people everywhere, the ...
073e22ec2b44a5f2f6289f09dc954ecd6ca6cbb4a4b33ffd88c3c150572569c0    Today, the Commission, in cooperation with the...
0a3650bef3282d09d971dd92dc01570b7ecbdb67a4bdd0ff1c8cf8443a06b937                                                  NaN
                                                                                          ...                        
fa793c764aaf89c283f747036c9d73fd66f0a7e3692be56b8ac985610bef2417    The EU Strategy on COVID-19 Therapeutics deliv...
fb9c4d1634da94c73fd529b44ec40057c28a5c925a8b8e71d73c49e5a815784d    COVID-19 is a severe public health emergency f...
fbad1d421cf80d3e0e4c6b6d3d60a2a6ec8bed4ee19e07cd15d9

In [17]:
# Create new dataframe with the wanted content and columns from EU timeline dataset
tmp_eu_timeline_df = pd.DataFrame(eu_timeline_df[["date", "title", "detail_content"]])
tmp_eu_timeline_df["doc_source"] = "eu_timeline"

In [18]:
# See the newly created EU timeline  dataframe
tmp_eu_timeline_df

,date,title,detail_content,doc_source
_id,,,,
01dc34ca5f6bfa5316012d6f89a4256b8d2ce6e3a12210b0406903dd15bf36ac,2021-01-28,EU countries adopt guidelines on proof of vacc...,NaN,eu_timeline
04d776c061243576ffc1e1c5a724d41ecca2ed12339b2ff473c5aa2ff1a071c7,2021-06-22,Commission endorses Latvia's €1.8 billion reco...,The European Commission has today adopted a po...,eu_timeline
04eb55005a803c7353e5d5ed339d7f3a72cc8da6e49f2b8b152bf8fb40a1398f,2020-06-16,Commission unveils EU vaccines strategy,"Today, to help protect people everywhere, the ...",eu_timeline
073e22ec2b44a5f2f6289f09dc954ecd6ca6cbb4a4b33ffd88c3c150572569c0,2020-04-15,European roadmap shows path towards common lif...,"Today, the Commission, in cooperation with the...",eu_timeline
0a3650bef3282d09d971dd92dc01570b7ecbdb67a4bdd0ff1c8cf8443a06b937,2020-12-18,Commission puts forward rules on rapid antigen...,NaN,eu_timeline
...,...,...,...,...
fa793c764aaf89c283f747036c9d73fd66f0a7e3692be56b8ac985610bef2417,2021-06-29,Commission identifies five promising candidate...,The EU Strategy on COVID-19 Therapeutics deliv...,eu_timeline
fb9c4d1634da94c73fd529b44ec40057c28a5c925a8b8e71d73c49e5a815784d,2020-03-13,Setting out coordinated response to counter th...,COVID-19 is a severe public health emergency f...,eu_timeline
fbad1d421cf80d3e0e4c6b6d3d60a2a6ec8bed4ee19e07cd15d963fa74352420,2021-01-28,Prolonging State aid Temporary Framework to fu...,The European Commission has adopted a,eu_timeline


In [19]:
# Print dataframe columns
ir_timeline_df.columns



Index(['keyword', 'page_type', 'page_link', 'department_data',
       'published_date', 'updated_date', 'title', 'content', 'content_links',
       'campaigns_links', 'part_of_links', 'documents'],
      dtype='object')

In [20]:
#What are the columns that shows dates
date_columns = [col for col in ir_timeline_df.columns if "date" in col]
ir_timeline_df[date_columns]



,published_date,updated_date
_id,,
41c0f398402f522980ec6e3ae49ef3b60b7139c1deb0ba75db830e77f62d5bdd,2021-03-24,2021-03-24
41d85d20ea2ffeaba4c27e36def4c03986ad83679fb62a07de6214abf0309ae1,2020-07-30,2020-07-30
420468c03e6550d8111344aa4d28d7e7cfd70037b16f8cab3451ff52d6bbd7b5,2020-06-06,2020-06-06
42081c8a3c63d62e4b0ef27f6a3386848b45d71665318ef14642649bb9b11b3e,2021-07-21,2021-07-23
421dd88edfd80b1fbc6c350565455e1a218774c9d4f1d74bbe2047d972187736,2020-12-10,2020-12-11
...,...,...
ff4ea2ef933287c4125d898245e878ae9fcb1b356b3c60de288dd97020bdd3a5,2021-01-22,2021-01-23
ff7508dca9aaf53565b08dc6826102bff9176584afc245b9947c18dae946506a,2021-08-25,2021-08-27
ff82338ae7c8129669562babc1c0d7a893a9c59f6d2bd00cefe853539721367e,2020-03-20,2020-03-20


In [21]:
# Create new dataframe with the wanted content and columns from Ireland timeline dataset
tmp_ir_timeline_df = pd.DataFrame(ir_timeline_df[["published_date", "title", "content"]])
tmp_ir_timeline_df["doc_source"] = "ireland_timeline"

In [22]:
# See the newly created Ireland timeline  dataframe
tmp_ir_timeline_df


,published_date,title,content,doc_source
_id,,,,
41c0f398402f522980ec6e3ae49ef3b60b7139c1deb0ba75db830e77f62d5bdd,2021-03-24,Statement by Minister for Education Norma Fole...,The State Examinations Commission is today iss...,ireland_timeline
41d85d20ea2ffeaba4c27e36def4c03986ad83679fb62a07de6214abf0309ae1,2020-07-30,From pencil and paper to drones and satellites...,July 31 2020 marks the 175th anniversary of th...,ireland_timeline
420468c03e6550d8111344aa4d28d7e7cfd70037b16f8cab3451ff52d6bbd7b5,2020-06-06,Return to Sport Expert Group recommends furthe...,"The Minister for Transport, Tourism and Sport,...",ireland_timeline
42081c8a3c63d62e4b0ef27f6a3386848b45d71665318ef14642649bb9b11b3e,2021-07-21,Government to put “special focus” on Drogheda ...,Government agrees that funding applications un...,ireland_timeline
421dd88edfd80b1fbc6c350565455e1a218774c9d4f1d74bbe2047d972187736,2020-12-10,Statement from the National Public Health Emer...,1. Hospital statistics 2. Gender of patients 3...,ireland_timeline
...,...,...,...,...
ff4ea2ef933287c4125d898245e878ae9fcb1b356b3c60de288dd97020bdd3a5,2021-01-22,Statement from the National Public Health Emer...,The Health Protection Surveillance Centre has ...,ireland_timeline
ff7508dca9aaf53565b08dc6826102bff9176584afc245b9947c18dae946506a,2021-08-25,Ministers McConalogue and Heydon launch Code o...,"The Minister for Agriculture, Food and the Mar...",ireland_timeline
ff82338ae7c8129669562babc1c0d7a893a9c59f6d2bd00cefe853539721367e,2020-03-20,Press Release on Civil Defence in the context ...,Civil Defence (Cosaint Shibhialta) is a statut...,ireland_timeline


In [23]:
def replace_non_english_content(text):
    if text is not None:
        language = detect_langs(text)
        language_details = str(language[0]).split(":")
        if language_details[0] == "en" and float(language_details[1]) > 0.95:
            return text
        else:
            return None




In [24]:
#Renaming columns in dataframes and dropping rows without content, title and date
data_frames = [tmp_cellar_df, tmp_eu_timeline_df, tmp_ir_timeline_df, tmp_pwdb_df]
for data_frame in data_frames:
    data_frame.columns = ["Date", "Title", "Content", "Document_source"]
    data_frame["Content"] = data_frame["Content"].apply(lambda x: x if x != "" else None)
    data_frame["Content"] = data_frame["Content"].replace({np.nan: None}).apply(lambda x: replace_non_english_content(x))
    data_frame.dropna(subset=['Content', 'Title', 'Date'], how="any", inplace=True)
    # data_frame.dropna(subset=['Title'], inplace=True)
    # data_frame.dropna(subset=['Date'], inplace=True)




In [25]:
#Unified datasets dataframe
unified_datasets_df = pd.DataFrame(pd.concat(data_frames))
unified_datasets_df




,Date,Title,Content,Document_source
_id,,,,
0e3d35dcad11d1d80f8fb1c35599be27fb3e864874ab1ac679fcd004723bc2e3,2020-01-30,Prior notification of a concentration (Case M....,30.1.2020 EN Official Journal of the European ...,eu_cellar
39dac43e4fffa3fc60906f3563e4d1547aea4a0f94e17cd28ec144b117e25df6,2021-08-30,COMMISSION STAFF WORKING DOCUMENT […] Accompan...,"EUROPEAN COMMISSION Brussels, 30.8.2021 SWD(20...",eu_cellar
00f65d205b8df943ae8acbd34b729c3da11da1646e7e324a6c517f95f950a5d9,2020-06-18,P9_TA(2020)0157 Amending Regulations (EU) No 5...,8.9.2021 EN Official Journal of the European U...,eu_cellar
002086bd15c9aba8b8b3cdf88498e25735cf66043ddab8877aca317256615aff,2020-09-07,Opinion No 6/2020 (pursuant to Article 287(4) ...,20.10.2020 EN Official Journal of the European...,eu_cellar
014589ceb95203c13ee4bd097ddf2f164657cb6aeb1650db280b2d8c7a57977d,2021-05-20,Council Recommendation (EU) 2021/816 20 May 20...,21.5.2021 EN Official Journal of the European ...,eu_cellar
...,...,...,...,...
tika/330d076853d0731e295d75f70841b445e4bc45061a8ae1507766fa26d778cc6f,2020-04-12,Government support to assist the sports sector...,"As sport events have been prohibited, the fina...",pwdb
tika/ffaa29a24828dd01c81d66ece0f8f3c21451083be7a0b6d2f4ed954b9378b0e6,2020-10-23,National public sector collective agreement 2021,"On 16 October 2020, the Government of the Repu...",pwdb
tika/3df93a8fb254187e8783d19760b465a98d88863e59a513b35a4d8062248e4190,2020-04-16,"Lifting of overtime work restrictions, extensi...",The government with a Legislative Act (Offici...,pwdb


In [27]:
#Upload to elastic
store_registry.es_index_store().put_dataframe(index_name=config.UNIFIED_DATASET_ELASTIC_SEARCH_INDEX_NAME,
                                              content=unified_datasets_df)


 98% (6116 of 6194) |################### | Elapsed Time: 0:00:00 ETA:   0:00:00

6194